## Exercise 114.

> Implement the Baby-JubJub twisted Edwards curve equation in Circom and compile it into an R1CS and associated witness generator.

```bash
$ circom ex114.circom --r1cs --wasm --sym --c
```

## Exercise 115. 

> Consider exercise 114 and execute a 3-party trusted setup phase for the baby- jubjub circuit.

```bash
$ snarkjs powersoftau new bn128 4 pot4_0000.ptau -v
$ snarkjs powersoftau contribute pot4_0000.ptau pot4_0001.ptau -name="1st_cont" -v
$ snarkjs powersoftau verify pot4_0001.ptau
$ snarkjs powersoftau beacon pot4_0001.ptau pot4_beacon.ptau 0102030405060708090a0b0c00 10 -n="Final Beacon"
$ snarkjs powersoftau prepare phase2 pot4_beacon.ptau pot4_final.ptau -v
$ snarkjs powersoftau verify pot4_final.ptau
$ snarkjs groth16 setup ex114.r1cs pot4_final.ptau ex114_check_baby_jubjub.zkey
```

## Exercise 116. 

> Consider exercise 114 and the CRS from exercise 115. Use Sage to generate a valid curve point on the Baby-JubJub curve in twisted Edwards form, that is not the neutral element and use this point as input to the circuit. Generate a witness for this input and use Snark.js to compute an instance as well as a zk-SNARK for this witness.

we know the Twisted Edwards Curve fulfills the following equation:

$$
\begin{align*}
    a \cdot x^2 + y^2 = 1 + d \cdot x^2 \cdot y^2
\end{align*}
$$
solving for $y$ we have that:

$$
\begin{align*}
    a \cdot x^2 - 1 &= d \cdot x^2 \cdot y^2 - y^2 \\
    a \cdot x^2 - 1 &= (d \cdot x^2  - 1) \cdot y^2 \\
    y^2 &= \frac{a \cdot x^2 - 1}{d \cdot x^2  - 1}
\end{align*}
$$

**solution:** get a random $x$ and solve for $y$

Sagecode:

In [14]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
F = GF(p)
a = F(168700)
d = F(168696)

class BabyJubJub():
    def __init__(self, x, y):
        self.__a = a
        self.__d = d
        self.x = F(x)
        self.y = F(y)
        assert self.__a * self.x^2 + self.y^2 == 1 + self.__d * self.x^2 * self.y^2

    def __str__(self):
        return f"({self.x}, {self.y})"

while True:
    x = F.random_element()
    y_square = (a * x^2 - 1) / (d * x^2 - 1)
    if y_square.is_square():
        y = y_square.sqrt()
        print(BabyJubJub(x, y))
        break

(16986086298120674819542782247846808825933108632118439353546377158497680851273, 1293999442195814645412834676310522907979985549006820516227435268300304500835)


the run the next command to generate the witness

```bash
$ cd ex114_js  
$ node generate_witness.js ex114.wasm ../ex114_witness.json ex114_witness.wtns
$ snarkjs groth16 prove ../ex114_check_baby_jubjub.zkey ex114_witness.wtns ../proof.json ../public.json
```

## Exercise 117. 

> Consider exercise 114, the CRS from exercise 115 as well as the instance and zk-SNARK from exercise 116 and verify the zk-SNARK against the instance.

Run the following commands:

```bash
$ cd .. 
$ snarkjs zkey export verificationkey ex114_check_baby_jubjub.zkey verification_key.json
$ snarkjs groth16 verify verification_key.json public.json proof.json
```